### メモリ上の実体はStorageオブジェクトが持つ

Tensorのデータはメモリ上では全て1次元配列として保存されており、その実体を管理しているのがStorageオブジェクト

In [1]:
import torch

In [4]:
a = torch.FloatTensor([[[1,2,3],[4,5,6]],[[0.1,0.2,0.3],[0.4,0.5,0.6]]])
print("a ", a)
print("a size", a.size())
a_storage = a.storage()
print(type(a_storage))
print(a_storage)

a  tensor([[[1.0000, 2.0000, 3.0000],
         [4.0000, 5.0000, 6.0000]],

        [[0.1000, 0.2000, 0.3000],
         [0.4000, 0.5000, 0.6000]]])
a size torch.Size([2, 2, 3])
<class 'torch.storage.TypedStorage'>
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
 0.10000000149011612
 0.20000000298023224
 0.30000001192092896
 0.4000000059604645
 0.5
 0.6000000238418579
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 12]


In [5]:
a[0,1,1] = 10
print(a_storage)

 1.0
 2.0
 3.0
 4.0
 10.0
 6.0
 0.10000000149011612
 0.20000000298023224
 0.30000001192092896
 0.4000000059604645
 0.5
 0.6000000238418579
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 12]


TensorはStorageに対するビューの役目を果たす。
TensorからStorage(1次元配列)へマッピングするために、Tensorではオフセットとストライドを持っている。

In [6]:
a_offset = a.storage_offset()
print("a_offset", a_offset)
a_stride = a.stride()
print("a_stride", a_stride)

a_offset 0
a_stride (6, 3, 1)


aの場合、オフセット：0, ストライド:(6,3,1)
Storageのindex = 0(offset) + 0次index * 6 + 1次index * 3 + 2次index * 1

Tensorにインデックスを使ってアクセスすると、Storageは同じでオフセット・ストライドのみが異なるTensorオブジェクトが生成される。つまり、メモリコピーは発生しない

In [7]:
print("a", a)
print("a size", a.size())
print(a.storage_offset(), a.stride())

a tensor([[[ 1.0000,  2.0000,  3.0000],
         [ 4.0000, 10.0000,  6.0000]],

        [[ 0.1000,  0.2000,  0.3000],
         [ 0.4000,  0.5000,  0.6000]]])
a size torch.Size([2, 2, 3])
0 (6, 3, 1)


In [8]:
b = a[1:]
print("b", b)
print("b size", b.size())

b tensor([[[0.1000, 0.2000, 0.3000],
         [0.4000, 0.5000, 0.6000]]])
b size torch.Size([1, 2, 3])


In [9]:
print(b.storage_offset(), b.stride())

6 (6, 3, 1)


In [10]:
c = b.squeeze(0).transpose(0, 1)
print("c", c)
print("c size", c.size())
print(c.storage())

c tensor([[0.1000, 0.4000],
        [0.2000, 0.5000],
        [0.3000, 0.6000]])
c size torch.Size([3, 2])
 1.0
 2.0
 3.0
 4.0
 10.0
 6.0
 0.10000000149011612
 0.20000000298023224
 0.30000001192092896
 0.4000000059604645
 0.5
 0.6000000238418579
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 12]


### transpose

In [11]:
x = torch.Tensor([[[0,1,2],[3,4,5],[6,7,8]],[[10,11,12],[13,14,15],[16,17,18]]])
print(x)
print(x.size())
print(x.storage())

tensor([[[ 0.,  1.,  2.],
         [ 3.,  4.,  5.],
         [ 6.,  7.,  8.]],

        [[10., 11., 12.],
         [13., 14., 15.],
         [16., 17., 18.]]])
torch.Size([2, 3, 3])
 0.0
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
 7.0
 8.0
 10.0
 11.0
 12.0
 13.0
 14.0
 15.0
 16.0
 17.0
 18.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 18]


In [12]:
y = x.transpose(0, 1)
print(y)
print(y.size())
print(y.storage())

tensor([[[ 0.,  1.,  2.],
         [10., 11., 12.]],

        [[ 3.,  4.,  5.],
         [13., 14., 15.]],

        [[ 6.,  7.,  8.],
         [16., 17., 18.]]])
torch.Size([3, 2, 3])
 0.0
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
 7.0
 8.0
 10.0
 11.0
 12.0
 13.0
 14.0
 15.0
 16.0
 17.0
 18.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 18]


In [13]:
w = x.transpose(1,2)
print(w)
print(w.size())

tensor([[[ 0.,  3.,  6.],
         [ 1.,  4.,  7.],
         [ 2.,  5.,  8.]],

        [[10., 13., 16.],
         [11., 14., 17.],
         [12., 15., 18.]]])
torch.Size([2, 3, 3])


In [14]:
u = x.transpose(0,2)
print(u)
print(u.size())

tensor([[[ 0., 10.],
         [ 3., 13.],
         [ 6., 16.]],

        [[ 1., 11.],
         [ 4., 14.],
         [ 7., 17.]],

        [[ 2., 12.],
         [ 5., 15.],
         [ 8., 18.]]])
torch.Size([3, 3, 2])


### contiguous

offsetとstrideによってTensorからStorageへマッピングされるので、メモリレイアウトは変更されない。

TensorからStorageへのマッピングがメモリの連続した領域へのアクセスなっているかどうかをチェックする→is_contiguous

連続している方が、CPU/GPUのキャッシュを効率的に使えるため、場合によっては、今チェックが必要になる。

In [15]:
a = torch.tensor([[1,2,3],[4,5,6]], dtype=torch.float32)
a_storage = a.storage()
print(a_storage)

a[1,1] = 10
print(a)

print(a_storage)

 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]
tensor([[ 1.,  2.,  3.],
        [ 4., 10.,  6.]])
 1.0
 2.0
 3.0
 4.0
 10.0
 6.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]


In [16]:
b = a[1:]
print(b)
print(b.storage_offset(), b.stride())

c = a.transpose(0, 1)
print(c)
print(c.storage_offset(), c.stride())

tensor([[ 4., 10.,  6.]])
3 (3, 1)
tensor([[ 1.,  4.],
        [ 2., 10.],
        [ 3.,  6.]])
0 (1, 3)


In [18]:
a[1,2] = 11
print(a)

tensor([[ 1.,  2.,  3.],
        [ 4., 10., 11.]])


In [19]:
print(b)

tensor([[ 4., 10., 11.]])


### cはtransposeでメモリ配列を不連続に参照している

In [20]:
print(a.is_contiguous(), b.is_contiguous(), c.is_contiguous())

True True False


In [21]:
a.storage()

 1.0
 2.0
 3.0
 4.0
 10.0
 11.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [22]:
c.storage()

 1.0
 2.0
 3.0
 4.0
 10.0
 11.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]

In [23]:
print("a", a)
print("c", c)

a tensor([[ 1.,  2.,  3.],
        [ 4., 10., 11.]])
c tensor([[ 1.,  4.],
        [ 2., 10.],
        [ 3., 11.]])


aの参照順序: 0->1->2->3->4->5 連続
cの参照順序: 0->3->1->4->2->5 不連続

### Cを参照順序でメモリレイアウトを連続にする→contigous()

In [25]:
print("c_strage:", c.storage())
print("c_is_contiguous : ", c.is_contiguous())
c_cont = c.contiguous()
print("c_cont_strage:", c_cont.storage())
print("c_cont_is_contiguous : ", c_cont.is_contiguous())
# メモリコピーが発生する

c_strage:  1.0
 2.0
 3.0
 4.0
 10.0
 11.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]
c_is_contiguous :  False
c_cont_strage:  1.0
 4.0
 2.0
 10.0
 3.0
 11.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 6]
c_cont_is_contiguous :  True


### contigous()以外でメモリコピーが発生する事象

明示的にメモリコピーを行う(=Storageを生成する)→ clone()メソッド

In [26]:
a_clone = a.clone()
print(a)
print(a_clone)
a_clone[0,0] = 100
print(a_clone)
print(a)

tensor([[ 1.,  2.,  3.],
        [ 4., 10., 11.]])
tensor([[ 1.,  2.,  3.],
        [ 4., 10., 11.]])
tensor([[100.,   2.,   3.],
        [  4.,  10.,  11.]])
tensor([[ 1.,  2.,  3.],
        [ 4., 10., 11.]])


booleanインデックスを使用する

In [27]:
a_filterd = a[a >= 10]
print(a_filterd)

a_filterd[0] = 100
print(a)

tensor([10., 11.])
tensor([[ 1.,  2.,  3.],
        [ 4., 10., 11.]])


値そのものの変更やもとのStorageサイズを変更するようなメソッドでもコピーが発生

1.transposeやviewはoffsetとstrideの変更で済むため、コピーは発生しない
2.末尾"_"のTensorメソッドはStorage内のメモリを上書きする(in-place)のため、コピーは発生しない 

In [28]:
a = torch.tensor([1,2,3])
b = torch.tensor([4,5,6])

# コピー発生
c = torch.pow(a, 2)
print(c.storage())


 1
 4
 9
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 3]


In [29]:
# コピー発生しない
c_ = c.pow_(2)
print(c_.storage())

print(c.storage())

 1
 16
 81
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 3]
 1
 16
 81
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 3]


In [30]:
# コピー発生
d = torch.cat((a,b))
print(d.storage())

 1
 2
 3
 4
 5
 6
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 6]


### GPUにデータを送るとメモリコピーになる

In [31]:
a = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32)

a_gpu = a.to(device="cuda")

print(a_gpu)
# tensor([[1., 2., 3.],
#         [4., 5., 6.]], device='cuda:0')

print(a_gpu.storage())
#  1.0
#  2.0
#  3.0
#  4.0
#  5.0
#  6.0
# [torch.cuda.FloatStorage of size 6]

tensor([[1., 2., 3.],
        [4., 5., 6.]], device='cuda:0')
 1.0
 2.0
 3.0
 4.0
 5.0
 6.0
[torch.storage.TypedStorage(dtype=torch.float32, device=cuda:0) of size 6]
